In [ ]:
%run "0a_Workspace_setup.ipynb"
#%run "0b_Create_poi_files.ipynb"

hru_gdf = create_hru_gdf(
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

#### Useful grouping functions for HRUs (Norton, P.A.)

In [ ]:
def subset_stream_network(dag_ds, uscutoff_seg, dsmost_seg):  # (from Bandit)
    """Extract subset of stream network

    :param dag_ds: Directed, acyclic graph of downstream stream network
    :param uscutoff_seg: List of upstream cutoff segments
    :param dsmost_seg: List of outlet segments to start extraction from

    :returns: Stream network of extracted segments
    """

    # taken from Bandit bandit_helpers.py

    # Create the upstream graph
    dag_us = dag_ds.reverse()

    # Trim the u/s graph to remove segments above the u/s cutoff segments
    try:
        for xx in uscutoff_seg:
            try:
                dag_us.remove_nodes_from(nx.dfs_predecessors(dag_us, xx))

                # Also remove the cutoff segment itself
                dag_us.remove_node(xx)
            except KeyError:
                print(f"WARNING: nhm_segment {xx} does not exist in stream network")
    except TypeError:
        print(
            "\nSelected cutoffs should at least be an empty list instead of NoneType."
        )

    # =======================================
    # Given a d/s segment (dsmost_seg) create a subset of u/s segments

    # Get all unique segments u/s of the starting segment
    uniq_seg_us: Set[int] = set()
    if dsmost_seg:
        for xx in dsmost_seg:
            try:
                pred = nx.dfs_predecessors(dag_us, xx)
                uniq_seg_us = uniq_seg_us.union(
                    set(pred.keys()).union(set(pred.values()))
                )
            except KeyError:
                print(f"KeyError: Segment {xx} does not exist in stream network")

        # Get a subgraph in the dag_ds graph and return the edges
        dag_ds_subset = dag_ds.subgraph(uniq_seg_us).copy()

        node_outlets = [ee[0] for ee in dag_ds_subset.edges()]
        true_outlets = set(dsmost_seg).difference(set(node_outlets))

        # Add the downstream segments that exit the subgraph
        for xx in true_outlets:
            nhm_outlet = list(dag_ds.neighbors(xx))[0]
            dag_ds_subset.add_node(
                nhm_outlet, style="filled", fontcolor="white", fillcolor="grey"
            )
            dag_ds_subset.add_edge(xx, nhm_outlet)
            dag_ds_subset.nodes[xx]["style"] = "filled"
            dag_ds_subset.nodes[xx]["fontcolor"] = "white"
            dag_ds_subset.nodes[xx]["fillcolor"] = "blue"
    else:
        # No outlets specified so pull the full model
        dag_ds_subset = dag_ds

    return dag_ds_subset


def hrus_by_seg(pdb, segs):  # (custom code)
    # segs: global segment IDs

    if isinstance(segs, int):
        segs = [segs]
    elif isinstance(segs, KeysView):
        segs = list(segs)

    seg_hrus = {}
    seg_to_hru = pdb.seg_to_hru

    # Generate stream network for the model
    dag_streamnet = pdb.stream_network()

    for cseg in segs:
        # Lookup segment for the current POI
        dsmost_seg = [cseg]

        # Get subset of stream network for given POI
        dag_ds_subset = subset_stream_network(dag_streamnet, set(), dsmost_seg)

        # Create list of segments in the subset
        toseg_idx = list(set(xx[0] for xx in dag_ds_subset.edges))

        # Build list of HRUs that contribute to the POI
        final_hru_list = []

        for xx in toseg_idx:
            try:
                for yy in seg_to_hru[xx]:
                    final_hru_list.append(yy)
            except KeyError:
                # print(f'Segment {xx} has no HRUs connected to it') # comment this out and add pass to not print the KeyError
                pass
        final_hru_list.sort()

        seg_hrus[cseg] = final_hru_list

    return seg_hrus


def hrus_by_poi(pdb, poi):  # (custom code)
    if isinstance(poi, str):
        poi = [poi]
    elif isinstance(poi, KeysView):
        poi = list(poi)

    poi_hrus = {}
    nhm_seg = pdb.get("nhm_seg").data
    pois_dict = pdb.poi_to_seg
    seg_to_hru = pdb.seg_to_hru

    # Generate stream network for the model
    dag_streamnet = pdb.stream_network()

    for cpoi in poi:
        # Lookup global segment id for the current POI
        dsmost_seg = [nhm_seg[pois_dict[cpoi] - 1]]

        # Get subset of stream network for given POI
        dag_ds_subset = subset_stream_network(dag_streamnet, set(), dsmost_seg)

        # Create list of segments in the subset
        toseg_idx = list(set(xx[0] for xx in dag_ds_subset.edges))

        # Build list of HRUs that contribute to the POI
        final_hru_list = []

        for xx in toseg_idx:
            try:
                for yy in seg_to_hru[xx]:
                    final_hru_list.append(yy)
            except KeyError:
                # Not all segments have HRUs connected to them
                # print(f'{cpoi}: Segment {xx} has no HRUs connected to it')
                pass
        final_hru_list.sort()
        poi_hrus[cpoi] = final_hru_list

    return poi_hrus

# Map HRU's, segments and POIs as folium.map elements

#### Set mapping extent and base map elements


In [ ]:
# Set approximate latitude, longitude and zoom level for subbasin is calculated for starting point of folium.map plot window.
pfile_lat = hru_gdf["hru_lat"].mean()
pfile_lon = hru_gdf["hru_lon"].mean()
zoom = 7

In [ ]:
# Set up a custom tile for base map

# This can be tricky with syntax but if you go to this link you will find resources that have options beyond the few defualt options in folium leaflet.
# http://leaflet-extras.github.io/leaflet-providers/preview/
# These tiles will also work in the minimap, but can get glitchy if the same tile var is used in the minimap and the main map child object.

USGStopo_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_topo",
    zoom_start=zoom,
    name="USGSTopo",
)
USGSHydroCached_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSHydroCached/MapServer/tile/{z}/{y}/{x}",
    attr="USGSHydroCached",
    zoom_start=zoom,
    name="USGSHydroCached",
)

In [ ]:
# Set up inset map
# This requires folium plugins. (from folium import plugins)
minimap = plugins.MiniMap(
    tile_layer="OpenStreetMap",
    # attr = 'USGS_topo',
    position="topleft",
    # zoom_level_offset=- 4,
    height=200,
    width=200,
    collapsed_height=25,
    collapsed_width=25,
    zoom_level_fixed=5,
    toggle_display=True,
    collapsed=True,
)

In [ ]:
# Set up referrence basin boundary shape file
crs = hru_gdf.crs
# admin_basin_df=gpd.read_file(root_dir / "OR_recharge_source_files/Boundaries/Oregon_Administrative_Areas_shapefile/owrd_basins.shp").to_crs(crs)
# #admin_basin_df["centroid"] = admin_basin_df.centroid# Opted for below so could adjust offset
# admin_basin_df['lon'] = admin_basin_df.centroid.x # copy over the reprojected longitude
# admin_basin_df['lat'] =admin_basin_df.centroid.y # copy over the reprojected latitude

# admin_basin_df_line = admin_basin_df.boundary

## 2. Create a Map of subbasin HRUs, segments, and POIs
This section will 
1) plot the HRUs as a folium map, 
2) add the stream segments as a folium map with a tooltip, and 
3) (optonal) plot the centroids of the HRUs as a circle marker with a popup to identify the associated stream segment in the model. 
This map will be useful for looking at HRU connections to stream segments, especially in areas where stream segments are scant and where flow is moved out of the model area.

#### Style Functions for mapping

In [ ]:
style_function_hru_map = lambda x: {
    "opacity": 1,
    "fillColor": "#00000000",  #'goldenrod',
    "color": "tan",
    "weight": 1.5,
}
highlight_function_hru_map = lambda x: {
    "opacity": 0.5,
    "color": "gray",
    "fillColor": "gray",
    "weight": 3,
}
style_function_seg_map = lambda x: {"opacity": 1, "color": "#217de7", "weight": 2}
highlight_function_seg_map = lambda x: {"opacity": 1, "color": "white", "weight": 4}
transparent = lambda x: {
    "fillColor": "#00000000",
    "color": "#00000000",
    "weight": 4,
}
# admin_basin_style = lambda x: {'fillColor': '#00000000',
#                                #'fill_opacity' : .8,
#                                'color': 'black',
#                                'weight': 2}

cp_style_function = lambda feature: {
    "fillColor": linear(par_sel_color_dict[feature["id"]]),
    "color": "tan",
    "weight": 1,
    # "dashArray": "5, 5",
    "fillOpacity": 0.3,
}

#### Make groups of HRUs for POIs

In [ ]:
# Create poi list from the POI dataframe
poi_list = poi_df["poi_gage_id"].values.tolist()

# Make a dictionary of pois and the list of HRUs in the contributing area for each poi
hru_poi_dict = hrus_by_poi(pdb, poi_list)  # Helper function from pyPRMS

# Sort the dictionary: this is important for the reverse dictionary (next step)
# to accurately give a poi_group to hrus that contribute to a downstream-gage
sorted_items = sorted(
    hru_poi_dict.items(), key=lambda item: -len(item[1])
)  # the - reverses the sorting order
hru_poi_dict = dict(sorted_items[:])

reversed_hru_poi_dict = {val: key for key in hru_poi_dict for val in hru_poi_dict[key]}

# assigns poi_group value to all hrus#Keep for later application
hru_gdf["poi_group"] = hru_gdf["nhm_id"].map(reversed_hru_poi_dict)

### Build plots at each POI that show par values of contributing HRUs

In [ ]:
%%time
# Builds plots, takes  20 minutes to build all param plots for all pois
cal_hru_params = nhru_params + nhru_nmonths_params
for idx, par in enumerate(cal_hru_params):
    try:
        pdb.get(par).dimensions["nmonths"].size

    except:
        print(f"Checking for {par} dimensioned by nhru.")

        for idx, poi_id in enumerate(poi_list):
            par_plot_file = Folium_maps_dir / f"{par}_{poi_id}.txt"
            if par_plot_file.exists():
                pass
                # print(f'{output_var_sel}_{poi_id} exists. To recreate the plot, remove the file from')
                # print(par_plot_file)
            else:

                ##%%time = par
                # Preporcessing: pulling only the selected param values for the HRUs related to the selected POI to plot.
                output_var_sel_plot_df = hru_gdf[
                    hru_gdf["nhm_id"].isin(hru_poi_dict[poi_id])
                ]
                output_var_sel_plot_df = output_var_sel_plot_df.sort_values(
                    ["hru_area"], ascending=True
                )
                output_var_sel_plot_df.hru_area = (
                    output_var_sel_plot_df.hru_area.round()
                )

                x_axis_var = "hru_area"  # we broke this out separately to quickly generate new plots based on a different variable for the x-axis
                fig = px.scatter(
                    output_var_sel_plot_df,
                    x=x_axis_var,
                    y=par,
                    # markers = True,
                    custom_data="nhm_id",
                    color="poi_group",
                    labels={"poi_group": "Downstream POI"},
                )

                fig.update_layout(
                    title=dict(
                        text=f"{par} for HRUs in {poi_id} catchment",
                        font=dict(size=18),
                        automargin=True,
                        yref="paper",
                    ),
                    width=500,
                    height=300,
                    showlegend=True,
                    # legend=dict(orientation="h",yanchor="bottom",y=1.02, xanchor="right", x=1),# this will put the legend above the plot
                    # margin=dict(l=20, r=20, t=20, b=20),#hesitate to customize until final plots are determined
                    font=dict(family="Arial", size=10, color="#7f7f7f"),  # font color
                    paper_bgcolor="linen",
                    plot_bgcolor="white",
                )

                fig.update_yaxes(title_text=f'{par}, {pdb.get(par).meta["units"]}')
                fig.update_xaxes(
                    title_text=f'{x_axis_var}, {pdb.get(x_axis_var).meta["units"]}'
                )

                fig.update_xaxes(
                    ticks="inside", tickwidth=2, tickcolor="black", ticklen=10
                )
                fig.update_yaxes(
                    ticks="inside", tickwidth=2, tickcolor="black", ticklen=10
                )

                # tick_step = np.round(((output_var_sel_plot_df.hru_area.max())*0.1)+500, -3)
                # fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0.0, dtick = tick_step))

                fig.update_xaxes(
                    showline=True,
                    linewidth=2,
                    linecolor="black",
                    showgrid=False,
                    # gridcolor='lightgrey',
                )
                fig.update_yaxes(
                    showline=True,
                    linewidth=2,
                    linecolor="black",
                    showgrid=False,
                    # gridcolor='lightgrey',
                )

                # fig.update_xaxes(type='category')
                fig.update_xaxes(autorange=True)

                fig.update_traces(
                    hovertemplate="<br>".join(
                        [
                            "parameter value: %{y}",
                            "nhu area: %{x}",
                            "hru: %{customdata[0]}",
                        ]
                    )
                )

                fig.update_layout(hovermode="closest")
                fig.update_layout(
                    hoverlabel=dict(
                        bgcolor="linen", font_size=13, font_family="Rockwell"
                    )
                )

                # Creating the html code for the plotly plot
                text_div = plotly.offline.plot(
                    fig, include_plotlyjs=False, output_type="div"
                )

                # Saving the plot as txt file with the html code
                # idx = 1
                with open(Folium_maps_dir / f"{par}_{poi_id}.txt", "w") as f:
                    f.write(text_div)

                # fig.show()

    else:
        print(f"Checking for {par} dimensioned by nhru and nmonths")

        for idx, poi_id in enumerate(poi_list):

            par_plot_file = Folium_maps_dir / f"{par}_{poi_id}.txt"
            if par_plot_file.exists():
                pass
                # print(f'{output_var_sel}_{poi_id} exists. To recreate the plot, remove the file from')
                # print(par_plot_file)
            else:
                # output_var_sel = par

                # Reshapes the monthly data for plotting: assigns a "month" number
                first = True
                for vv in range(1, 13):
                    if first:
                        zz = f"{par}_{str(vv)}"
                        df = hru_gdf[["nhm_id", zz]]
                        df["month"] = vv
                        df[par] = df[zz]
                        df.drop(columns=zz, inplace=True)
                        first = False
                    else:
                        zz = f"{par}_{str(vv)}"
                        df2 = hru_gdf[["nhm_id", zz]]
                        df2["month"] = vv
                        df2[par] = df2[zz]
                        df2.drop(columns=zz, inplace=True)

                        df = pd.concat([df, df2], ignore_index=True)

                nhru_params_nmonths_sel_df = df.copy()
                ############################################################################################
                nhru_params_nmonths_sel_plot_df = nhru_params_nmonths_sel_df[
                    nhru_params_nmonths_sel_df["nhm_id"].isin(hru_poi_dict[poi_id])
                ]
                # nhru_params_nmonths_sel_plot_df = nhru_params_nmonths_sel_df[nhru_params_nmonths_sel_df['poi_group'] == poi_id]

                fig = px.line(
                    nhru_params_nmonths_sel_plot_df,
                    x="month",
                    y=par,
                    markers=True,
                    custom_data=nhru_params_nmonths_sel_plot_df[["nhm_id"]],
                    color="nhm_id",
                    labels={"nhm_id": "HRU"},
                )

                fig.update_layout(
                    title_text=f"{par} for HRUs in {poi_id} catchment",
                    width=500,
                    height=300,
                    showlegend=True,
                    # legend=dict(orientation="h",yanchor="bottom",y=1.02, xanchor="right", x=1),
                    font=dict(family="Arial", size=10, color="#7f7f7f"),  # font color
                    paper_bgcolor="linen",
                    plot_bgcolor="white",
                )

                fig.update_yaxes(title_text=f"{par}, units")
                fig.update_xaxes(title_text="Months")

                fig.update_xaxes(
                    ticks="inside", tickwidth=2, tickcolor="black", ticklen=10
                )
                fig.update_yaxes(
                    ticks="inside", tickwidth=2, tickcolor="black", ticklen=10
                )

                fig.update_xaxes(
                    showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
                )
                fig.update_yaxes(
                    showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
                )

                fig.update_xaxes(autorange=True)

                fig.update_traces(
                    hovertemplate="<br>".join(
                        [
                            "{parameter: %{y}",
                            "month: %{x}",
                            "hru: %{customdata[0]}",
                        ]
                    )
                )

                fig.update_layout(hovermode="closest")
                fig.update_layout(
                    hoverlabel=dict(
                        bgcolor="linen", font_size=13, font_family="Rockwell"
                    )
                )

                # Creating the html code for the plotly plot
                text_div = plotly.offline.plot(
                    fig, include_plotlyjs=False, output_type="div"
                )

                # Saving the plot as txt file with the html code
                with open(Folium_maps_dir / f"{par}_{poi_id}.txt", "w") as f:
                    f.write(text_div)

                # fig.show()

# Now select a parameter to view on the map

In [ ]:
from ipywidgets import widgets

par_sel = cal_hru_params[0]
# sel_flag = False

v = widgets.Dropdown(
    options=cal_hru_params,
    value=cal_hru_params[0],
    description="Select a parameter to view in plots:",
)


def on_change(change):
    global par_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        par_sel = v.value
        # sel_flag = True


v.observe(on_change)

display(v)

In [ ]:
# After selecting from the dropdown above, click here, then go to Run in the Menue

In [ ]:
mo_num_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

mo_names = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
mo_name = "July"  # set default value

try:
    pdb.get(par_sel).dimensions["nmonths"].size

except:
    print(f"{par_sel} dimensioned by nhru only.")

else:
    print(f"{par_sel} dimensioned by nhru and nmonths")
    mo_name = "July"  # set default value
    mo_sel = mo_num_dict[mo_name]

    m = widgets.Dropdown(
        options=mo_names,
        value=mo_names[6],  # set default value
        description="Select a month to display on the map:",
    )

    def on_change(change):
        global mo_sel, mo_name, mo_num_dict
        if change["type"] == "change" and change["name"] == "value":
            mo_name = m.value
            mo_sel = mo_num_dict[mo_name]

    m.observe(on_change)

    display(m)

In [ ]:
# After selecting from the dropdown above, click here, then go to Run in the Menue

In [ ]:
pdb.get(par_sel).meta["help"]

In [ ]:
# Creates a map with embedded plots of the slected parameter

m3 = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)
folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_Topo",
    zoom_start=zoom,
    name="USGS Topography",
    show=False,
).add_to(m3)

if par_sel in nhru_params:

    mo_sel = None  # set to none, this is not a monthly param
    hru_gdf_copy = hru_gdf.copy()
    hru_gdf_copy["nhm_id"] = hru_gdf_copy["nhm_id"].astype(str)
    hru_gdf_copy["hru_segment_nhm"] = hru_gdf_copy["hru_segment_nhm"].astype(str)

    hru_gdf_copy.set_index("nhm_id", inplace=True, drop=False)

    par_subset_df = hru_gdf.loc[:, ["nhm_id", par_sel]]
    par_subset_df["nhm_id"] = par_subset_df["nhm_id"].astype(str)
    par_subset_df.rename(columns={f"{par_sel}": "par_value"}, inplace=True)
    par_subset_df["par_value"] = np.round(par_subset_df["par_value"], 4)
    par_subset_df.set_index("nhm_id", inplace=True, drop=False)

    value_min = np.round(par_subset_df["par_value"].min(), 8)
    value_max = np.round(par_subset_df["par_value"].max(), 8)

    par_sel_color_dict = pd.Series(
        par_subset_df.par_value.values, index=par_subset_df.nhm_id
    ).to_dict()

    # Making par_bins
    sdv = par_subset_df["par_value"].std()
    mean = par_subset_df["par_value"].mean()

    par_bins = [
        value_min,
        np.round(value_min + (0.25 * (mean - value_min)), 5),
        np.round(value_min + (0.50 * (mean - value_min)), 5),
        np.round(value_min + (0.75 * (mean - value_min)), 5),
        np.round(mean, 3),
        np.round(value_max - (0.75 * (value_max - mean)), 5),
        np.round(value_max - (0.50 * (value_max - mean)), 5),
        np.round(value_max - (0.25 * (value_max - mean)), 5),
        value_max,
    ]

    #################################################
    if value_min != value_max:
        fig, ax = plt.subplots(figsize=(18, 0.5))
        fig.subplots_adjust(bottom=0.5)

        cmap = mplib.colors.ListedColormap(
            [
                "#8B0000",
                "#AC4800",
                "#CD9100",
                "#EEDA00",
                "#DADA13",
                "#91913B",
                "#484863",
                "#00008B",
            ]
        )
        cmap.set_over("0.25")
        cmap.set_under("0.75")

        bounds = par_bins
        norm = mplib.colors.BoundaryNorm(bounds, cmap.N)
        cb2 = mplib.colorbar.ColorbarBase(
            ax,
            cmap=cmap,
            norm=norm,
            boundaries=[0] + bounds + [13],
            extend=None,
            ticks=bounds,
            spacing="uniform",
            orientation="horizontal",
            alpha=0.45,
        )
        cb2.set_label(f'Discrete {par_sel} intervals, {pdb.get(par_sel).meta["units"]}')

        fig.set_facecolor("lightgray")
        fig.show()

    #######################################################

    if value_min == value_max:
        linear = cm.StepColormap(
            colors=[
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
            ],
            index=par_bins,
            vmin=0.00,
            vmax=0.05,
            caption="Total Standard deviation at the point[mm]",
            # tick_labels= ('0.01', '0.02', '0.03', '0.04')
        )
    else:
        linear = cm.StepColormap(
            colors=[
                "#8B0000",
                "#AC4800",
                "#CD9100",
                "#EEDA00",
                "#DADA13",
                "#91913B",
                "#484863",
                "#00008B",
            ],
            index=par_bins,
            vmin=0.00,
            vmax=0.05,
            caption="Total Standard deviation at the point[mm]",
            # tick_labels= ('0.01', '0.02', '0.03', '0.04')
        )
    popup_hru = folium.GeoJsonPopup(
        fields=["nhm_id", "hru_segment_nhm", par_sel],
        aliases=["hru", " flows to segment", f"{par_sel}"],
        labels=True,
        localize=True,
        style=(
            "font-size: 16px;"
        ),  # Note that this tooltip style sets the style for all tool_tips.
        # background-color: #F0EFEF;border: 2px solid black;font-family: arial; padding: 10px; background-color: #F0EFEF;
    )
    hru_map = folium.GeoJson(
        hru_gdf_copy,
        style_function=cp_style_function,  # style_function_hru_map,
        highlight_function=highlight_function_hru_map,
        name="NHM HRUs",
        popup=popup_hru,
        z_index_offset=40002,
    ).add_to(m3)

    # tooltip_hru=folium.GeoJsonTooltip(fields= ["nhm_id","hru_segment_nhm",par_sel],
    #                               aliases=["HRU"," flows to segment", f"{par_sel}"],
    #                               localize=False,
    #                               style=("background-color: #F0EFEF;border: 2px solid black;font-family: arial; font-size: 16px; padding: 10px;"),
    #                                  )


else:
    # mo_sel = '5'
    par_mo_sel = f"{par_sel}_{mo_sel}"
    value_min = hru_gdf[par_mo_sel].min()
    value_max = hru_gdf[par_mo_sel].max()

    hru_gdf_copy = hru_gdf.copy()
    hru_gdf_copy["nhm_id"] = hru_gdf_copy["nhm_id"].astype(str)
    hru_gdf_copy["hru_segment_nhm"] = hru_gdf_copy["hru_segment_nhm"].astype(str)

    hru_gdf_copy.set_index("nhm_id", inplace=True, drop=False)

    par_subset_df = hru_gdf.loc[:, ["nhm_id", par_mo_sel]]
    par_subset_df["nhm_id"] = par_subset_df["nhm_id"].astype(str)
    par_subset_df.rename(columns={f"{par_mo_sel}": "par_value"}, inplace=True)

    value_min = np.round(par_subset_df["par_value"].min(), 8)
    value_max = np.round(par_subset_df["par_value"].max(), 8)

    par_sel_color_dict = pd.Series(
        par_subset_df.par_value.values, index=par_subset_df.nhm_id
    ).to_dict()

    # Making par_bins
    sdv = par_subset_df["par_value"].std()
    mean = par_subset_df["par_value"].mean()

    par_bins = [
        value_min,
        np.round(value_min + (0.25 * (mean - value_min)), 5),
        np.round(value_min + (0.50 * (mean - value_min)), 5),
        np.round(value_min + (0.75 * (mean - value_min)), 5),
        np.round(mean, 3),
        np.round(value_max - (0.75 * (value_max - mean)), 5),
        np.round(value_max - (0.50 * (value_max - mean)), 5),
        np.round(value_max - (0.25 * (value_max - mean)), 5),
        value_max,
    ]

    #################################################
    if value_min != value_max:
        fig, ax = plt.subplots(figsize=(18, 0.5))
        fig.subplots_adjust(bottom=0.5)

        cmap = mplib.colors.ListedColormap(
            [
                "#8B0000",
                "#AC4800",
                "#CD9100",
                "#EEDA00",
                "#DADA13",
                "#91913B",
                "#484863",
                "#00008B",
            ]
        )
        cmap.set_over("0.25")
        cmap.set_under("0.75")

        bounds = par_bins
        norm = mplib.colors.BoundaryNorm(bounds, cmap.N)
        cb2 = mplib.colorbar.ColorbarBase(
            ax,
            cmap=cmap,
            norm=norm,
            boundaries=[0] + bounds + [13],
            extend=None,
            ticks=bounds,
            spacing="uniform",
            orientation="horizontal",
            alpha=0.45,
        )
        cb2.set_label(
            f'Discrete {par_sel} intervals, {pdb.get(par_sel).meta["units"]}'
        )  # {pdb.get(par_sel).units}

        fig.set_facecolor("lightgray")
        fig.show()

    #######################################################
    if value_min == value_max:
        linear = cm.StepColormap(
            colors=[
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
                "#000000",
            ],
            index=par_bins,
            vmin=0.00,
            vmax=0.05,
            caption="Total Standard deviation at the point[mm]",
            # tick_labels= ('0.01', '0.02', '0.03', '0.04')
        )
    else:
        linear = cm.StepColormap(
            colors=[
                "#8B0000",
                "#AC4800",
                "#CD9100",
                "#EEDA00",
                "#DADA13",
                "#91913B",
                "#484863",
                "#00008B",
            ],
            index=par_bins,
            vmin=0.00,
            vmax=0.05,
            caption="Total Standard deviation at the point[mm]",
            # tick_labels= ('0.01', '0.02', '0.03', '0.04')
        )
    popup_hru = folium.GeoJsonPopup(
        fields=["nhm_id", "hru_segment_nhm", par_mo_sel],
        aliases=["hru", " flows to segment", f"{par_sel} for {mo_name}"],
        labels=True,
        localize=False,
        style=(
            "font-size: 16px;"
        ),  # Note that this tooltip style sets the style for all tool_tips.
        # background-color: #F0EFEF;border: 2px solid black;font-family: arial; padding: 10px; background-color: #F0EFEF;
    )
    hru_map = folium.GeoJson(
        hru_gdf_copy,
        style_function=cp_style_function,  # style_function_hru_map,
        highlight_function=highlight_function_hru_map,
        name="NHM HRUs",
        popup=popup_hru,
        z_index_offset=40002,
    ).add_to(m3)
    # tooltip_hru=folium.GeoJsonTooltip(fields= ["nhm_id","hru_segment_nhm",par_mo_sel],
    #                               aliases=["HRU"," flows to segment", f"{par_sel} for {mo_name}"],
    #                               labels=True,
    #                               localize=False,
    #                               style=("background-color: #F0EFEF;border: 2px solid black;font-family: arial; font-size: 16px; padding: 10px;"),
    #                                  )

    # linear = cm.LinearColormap(["red", "yellow", "blue"], vmin= hru_gdf[par_mo_sel].min(), vmax=hru_gdf[par_mo_sel].max())

    # hru_map = folium.GeoJson(hru_gdf_copy,
    #                      style_function = cp_style_function,#style_function_hru_map,
    #                      highlight_function = highlight_function_hru_map,
    #                      name = "NHM HRUs",
    #                      z_index_offset = 40002).add_to(m3)
    # tooltip_hru=folium.GeoJsonTooltip(fields= ["nhm_id","hru_segment_nhm",par_mo_sel],
    #                               aliases=["HRU"," flows to segment", "par_value"],
    #                               labels=True)
# Add tool tip to map
# hru_map.add_child(tooltip_hru)


# Create and add segments map
popup_seg = folium.GeoJsonPopup(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
    localize=False,
)
seg_map = folium.GeoJson(
    seg_gdf,
    style_function=style_function_seg_map,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
    control=True,
    z_index_offset=40003,
    popup=popup_seg,
).add_to(m3)

# tooltip_seg=folium.GeoJsonTooltip(fields= ["model_idx","nhm_seg",
#                                           "tosegment_nhm"],
#                                   aliases=["local index", "Segment",
#                                           "flows to segment"],
#                                   labels=True)
# seg_map.add_child(tooltip_seg)


# add POI markers

marker_cluster = MarkerCluster(
    name="All the POIs",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=5000,
)
marker_cluster_label_poi = MarkerCluster(
    name="All the POI labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=4004,
)

marker_cluster_label_hru = MarkerCluster(
    name="All HRU labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=9,
    z_index_offset=4005,
)


m3.add_child(minimap)

for idx, row in poi_df.iterrows():
    poi_id = row["poi_id"]
    # Read ploty plot of each poi
    with open(Folium_maps_dir / f"{par_sel}_{poi_id}.txt", "r") as f:
        div_txt = f.read()

    # Create html code to insert the plotly plot to the folium pop up
    html = (
        """
    <html>
    <head>
         <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    </head>
    <body>
         <!-- Output from the Python script above: -->"""
        + div_txt
        + """</body>
    </html>"""
    )

    # Add the Plots to the popup
    iframe = folium.IFrame(html=html, width=845, height=475)
    # popup = folium.Popup(iframe, max_width=3250,parse_html=True)

    marker = folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        name=row["poi_id"],
        popup=folium.Popup(iframe, max_width=550, max_height=350, parse_html=True),
        radius=4,
        weight=2,
        color="black",
        fill=True,
        fill_color="Yellow",
        fill_opacity=1.0,
        draggable=True,
        z_index_offset=4006,
    ).add_to(marker_cluster)

    # marker_cluster.add_child(marker)
    text = f'{row["poi_id"]}'
    label_lat = row["latitude"] - 0.01
    label_lon = row["longitude"]

    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        z_index_offset=4007,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
        ),
    ).add_to(marker_cluster_label_poi)


for idx, row in hru_gdf.iterrows():
    text = f'{row["nhm_id"]}'
    label_lat = row["hru_lat"]
    label_lon = row["hru_lon"]
    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        z_index_offset=4008,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html='<div style="font-family: verdona; font-size: 10pt; font-weight: bold; color: black; text-shadow: 1px 1px 2px white;">%s</div>'
            % text,
        ),
    ).add_to(marker_cluster_label_hru)


marker_cluster.add_to(m3)
marker_cluster_label_poi.add_to(m3)
marker_cluster_label_hru.add_to(m3)

plugins.Fullscreen(position="topleft").add_to(m3)


folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

con.print(f"")
con.print(f"")
con.print(f"")
if mo_sel is None:
    con.print(f"NHM parameter {par_sel}", style="u bold black")
    con.print(f'{pdb.get(par_sel).meta["help"]}.', style="black")
else:
    con.print(f"NHM parameter {par_sel}", style="u bold black")
    con.print(
        f'{pdb.get(par_sel).meta["help"]}. The {par_sel} is a monthly parameter; map is displaying {mo_name} values.',
        style="black",
    )
if value_min == value_max:
    con.print(
        f"{par_sel} does not vary with a value of {np.round(value_min,3)}",
        style="black",
    )
# display(linear)


# con.print(f'{pdb.parameters.get(par_sel).description}.')

m3